In [1]:
import pandas as pd
import numpy as np
import pm4py
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [2]:
# Import the data
bpi_data = pd.read_csv("BPI_Challenge_2017.csv", sep=',', quotechar='"')
bpi_data.columns = bpi_data.columns.str.strip()
bpi_data['time'] = pd.to_datetime(bpi_data['time'], format='%Y/%m/%d %H:%M:%S.%f')
bpi_log = pm4py.format_dataframe(bpi_data, case_id='case', activity_key='event', timestamp_key='time')
display(bpi_log)

,case,event,time,lifecycle:transition,ApplicationType,LoanGoal,RequestedAmount,MonthlyCost,org:resource,Selected,...,Accepted,CreditScore,NumberOfTerms,EventOrigin,OfferedAmount,case:concept:name,concept:name,time:timestamp,@@index,@@case_index
0,Application_1000086665,A_Create Application,2016-08-03 17:57:21.673000+00:00,COMPLETE,New credit,"Other, see explanation",5000.0,NaN,User_1,NaN,...,NaN,NaN,NaN,Application,NaN,Application_1000086665,A_Create Application,2016-08-03 17:57:21.673000+00:00,0,0
1,Application_1000086665,A_Submitted,2016-08-03 17:57:21.734000+00:00,COMPLETE,New credit,"Other, see explanation",5000.0,NaN,User_1,NaN,...,NaN,NaN,NaN,Application,NaN,Application_1000086665,A_Submitted,2016-08-03 17:57:21.734000+00:00,1,0
2,Application_1000086665,W_Handle leads,2016-08-03 17:57:21.963000+00:00,SCHEDULE,New credit,"Other, see explanation",5000.0,NaN,User_1,NaN,...,NaN,NaN,NaN,Workflow,NaN,Application_1000086665,W_Handle leads,2016-08-03 17:57:21.963000+00:00,2,0
3,Application_1000086665,W_Handle leads,2016-08-03 17:58:28.286000+00:00,WITHDRAW,New credit,"Other, see explanation",5000.0,NaN,User_1,NaN,...,NaN,NaN,NaN,Workflow,NaN,Application_1000086665,W_Handle leads,2016-08-03 17:58:28.286000+00:00,3,0
4,Application_1000086665,W_Complete application,2016-08-03 17:58:28.293000+00:00,SCHEDULE,New credit,"Other, see explanation",5000.0,NaN,User_1,NaN,...,NaN,NaN,NaN,Workflow,NaN,Application_1000086665,W_Complete application,2016-08-03 17:58:28.293000+00:00,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202262,Application_999993812,W_Call incomplete files,2016-10-20 10:19:28.812000+00:00,RESUME,New credit,Caravan / Camper,30000.0,NaN,User_41,NaN,...,NaN,NaN,NaN,Workflow,NaN,Application_999993812,W_Call incomplete files,2016-10-20 10:19:28.812000+00:00,1202262,31508
1202263,Application_999993812,W_Call incomplete files,2016-10-20 10:21:59.667000+00:00,SUSPEND,New credit,Caravan / Camper,30000.0,NaN,User_41,NaN,...,NaN,NaN,NaN,Workflow,NaN,Application_999993812,W_Call incomplete files,2016-10-20 10:21:59.667000+00:00,1202263,31508
1202264,Application_999993812,O_Accepted,2016-10-24 08:24:30.056000+00:00,COMPLETE,New credit,Caravan / Camper,30000.0,NaN,User_68,NaN,...,NaN,NaN,NaN,Offer,NaN,Application_999993812,O_Accepted,2016-10-24 08:24:30.056000+00:00,1202264,31508
1202265,Application_999993812,A_Pending,2016-10-24 08:24:30.059000+00:00,COMPLETE,New credit,Caravan / Camper,30000.0,NaN,User_68,NaN,...,NaN,NaN,NaN,Application,NaN,Application_999993812,A_Pending,2016-10-24 08:24:30.059000+00:00,1202265,31508


In [13]:
import pandas as pd

# Load the dataset
final_df = pd.read_csv("case_durations_for_powerbi.csv")

# Create quantile bins (still based on minutes)
quantile_bins = pd.qcut(final_df['case_duration_minutes'], q=5)
bin_intervals = quantile_bins.cat.categories

# Format the bin labels in days
bin_labels_days = [
    f"{round(i.left / 1440, 1)} – {round(i.right / 1440, 1)} days"
    for i in bin_intervals
]

# Map intervals to new labels and order
label_mapping = dict(zip(bin_intervals, bin_labels_days))
order_mapping = {interval: order for order, interval in enumerate(bin_intervals)}

# Apply to dataframe
final_df['duration_bin'] = quantile_bins.map(label_mapping)
final_df['duration_order'] = quantile_bins.map(order_mapping)
final_df['LoanOutcome'] = final_df['loan_success'].map({1: "Success", 0: "Fail"})

# Save for Power BI
final_df.to_csv("case_durations_binned_days.csv", index=False)


In [ ]:
import pandas as pd

# Load data
bpi_data = pd.read_csv("BPI_Challenge_2017.csv", sep=',', quotechar='"')
bpi_data.columns = bpi_data.columns.str.strip()
bpi_data['time'] = pd.to_datetime(bpi_data['time'], format='%Y/%m/%d %H:%M:%S.%f')

# Flag successful applications
success_cases = bpi_data[bpi_data['event'] == 'A_Pending']['case'].unique()
bpi_data['loan_success'] = bpi_data['case'].isin(success_cases).astype(int)

# Flag in-person submissions
submitted_in_person_cases = bpi_data[bpi_data['event'] == 'A_Submitted']['case'].unique()
bpi_data['submitted_in_person'] = bpi_data['case'].isin(submitted_in_person_cases).astype(int)

# Drop duplicates to one row per case
case_level = bpi_data[['case', 'loan_success', 'submitted_in_person']].drop_duplicates('case')

# Readable labels
case_level['LoanOutcome'] = case_level['loan_success'].map({1: 'Success', 0: 'Fail'})
case_level['SubmissionMethod'] = case_level['submitted_in_person'].map({1: 'In Person', 0: 'Not In Person'})

# Export for Power BI
case_level.to_csv("submission_vs_success.csv", index=False)


In [ ]:
import pandas as pd

# Load and prepare the data
bpi_data = pd.read_csv("BPI_Challenge_2017.csv", sep=',', quotechar='"')
bpi_data.columns = bpi_data.columns.str.strip()
bpi_data['time'] = pd.to_datetime(bpi_data['time'], format='%Y/%m/%d %H:%M:%S.%f')

# Compute case duration
case_times = bpi_data.groupby('case')['time'].agg(['min', 'max'])
case_times['case_duration_minutes'] = (case_times['max'] - case_times['min']).dt.total_seconds() / 60

# Determine if case was successful (contains A_Pending event)
success_cases = bpi_data[bpi_data['event'] == 'A_Pending']['case'].unique()
case_times['loan_success'] = case_times.index.isin(success_cases).astype(int)

# Add LoanGoal to each case by taking first non-null LoanGoal per case
loan_goals = bpi_data[['case', 'LoanGoal']].dropna().drop_duplicates('case')
case_summary = case_times.reset_index().merge(loan_goals, on='case', how='left')

# Clean labels
case_summary['LoanOutcome'] = case_summary['loan_success'].map({1: 'Success', 0: 'Fail'})

# Export to CSV
case_summary.to_csv("loangoal_outcomes.csv", index=False)
